# Logic Apps as MCP Server: Enterprise Tool Integration

> **Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub
> **Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)
> **Copyright 2025 Ozgur Guler. All rights reserved.**

---

## What is MCP (Model Context Protocol)?

**MCP** is an open standard that enables AI agents to discover and invoke external tools. Think of it as **"USB-C for AI agents"** - a universal protocol for LLMs to access enterprise resources securely.

### Why MCP + Logic Apps?

Azure Logic Apps provides **1,400+ prebuilt connectors** to enterprise systems. By exposing Logic Apps as MCP servers, you give your AI agents access to:

| Category | Connectors | Examples |
|----------|------------|----------|
| **Enterprise Systems** | SAP, Dynamics 365, ServiceNow | Create orders, update tickets, sync data |
| **Productivity** | Office 365, SharePoint, Teams | Send emails, create documents, post messages |
| **Databases** | SQL Server, Oracle, CosmosDB | Query data, insert records, run procedures |
| **Cloud Services** | Azure, AWS, Google Cloud | Provision resources, manage infrastructure |
| **Communication** | Twilio, SendGrid, Slack | Send SMS, emails, notifications |
| **Custom APIs** | HTTP, OpenAPI, SOAP | Call any REST/SOAP endpoint |

---

## Architecture: How It Works

```
┌─────────────────────────────────────────────────────────────────────────────────┐
│                                USER REQUEST                                      │
│                "Create a support ticket for login issue"                         │
└─────────────────────────────────────────────────────────────────────────────────┘
                                       │
                                       ▼
┌─────────────────────────────────────────────────────────────────────────────────┐
│                         AZURE AI FOUNDRY AGENT                                   │
│  ┌───────────────────────────────────────────────────────────────────────────┐  │
│  │  1. Agent receives request                                                 │  │
│  │  2. Discovers available MCP tools from Logic Apps                         │  │
│  │  3. Selects appropriate tool (e.g., "CreateTicket")                       │  │
│  │  4. Generates tool call with parameters                                    │  │
│  └───────────────────────────────────────────────────────────────────────────┘  │
└─────────────────────────────────────────────────────────────────────────────────┘
                                       │
                            MCP Protocol (JSON-RPC)
                                       │
                                       ▼
┌─────────────────────────────────────────────────────────────────────────────────┐
│                     LOGIC APPS MCP SERVER (Standard)                             │
│  ┌───────────────────────────────────────────────────────────────────────────┐  │
│  │  MCP Endpoint: /api/mcpservers/{name}/mcp                                  │  │
│  │                                                                            │  │
│  │  Available Tools:                                                          │  │
│  │  • CreateTicket    → ServiceNow/Jira workflow                             │  │
│  │  • SendEmail       → Office 365 workflow                                   │  │
│  │  • UpdateCRM       → Salesforce/Dynamics workflow                          │  │
│  │  • QueryDatabase   → SQL Server workflow                                   │  │
│  └───────────────────────────────────────────────────────────────────────────┘  │
└─────────────────────────────────────────────────────────────────────────────────┘
                                       │
                                       ▼
┌─────────────────────────────────────────────────────────────────────────────────┐
│                      1,400+ ENTERPRISE CONNECTORS                                │
│                                                                                  │
│   ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐         │
│   │   SAP    │  │Salesforce│  │ServiceNow│  │   SQL    │  │Office 365│         │
│   └──────────┘  └──────────┘  └──────────┘  └──────────┘  └──────────┘         │
│   ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌──────────┐         │
│   │SharePoint│  │  Teams   │  │  Oracle  │  │  Twilio  │  │   AWS    │         │
│   └──────────┘  └──────────┘  └──────────┘  └──────────┘  └──────────┘         │
│                                                                                  │
└─────────────────────────────────────────────────────────────────────────────────┘
```

---

## Logic Apps Agent Workflow Types

Logic Apps supports two types of AI agent workflows:

| Type | Trigger | Interaction | Use Case |
|------|---------|-------------|----------|
| **Autonomous Agent** | Any trigger (HTTP, Schedule, Event) | No human interaction | Automated tasks, batch processing |
| **Conversational Agent** | "When chat session starts" | Human chat interface | Customer support, helpdesk |

### Agent Loop Process

```
┌─────────────────────────────────────────────────────────────────┐
│                    AGENT LOOP (Iterative)                        │
├─────────────────────────────────────────────────────────────────┤
│  1. Receive input (prompt, trigger data)                         │
│  2. LLM analyzes and plans required actions                      │
│  3. Select tools from available workflows                        │
│  4. Execute tool (Logic Apps workflow)                           │
│  5. Process results                                              │
│  6. Decide: More tools needed? → Loop back to step 2            │
│  7. Generate final response                                      │
└─────────────────────────────────────────────────────────────────┘
```

---

## Prerequisites

1. **Logic Apps Standard** app with MCP enabled (see Section 5 for setup)
2. **Workflow** with Request trigger exposed as MCP tool
3. **MCP Server URL**: `https://<logic-app>.azurewebsites.net/api/mcpservers/<name>/mcp`
4. **Azure AI Foundry** project with agent capabilities

### Authentication Options

| Method | Use Case | Configuration |
|--------|----------|---------------|
| **Developer Key** | Development/Testing | Auto-injected by portal |
| **Agent Identity Token** | Production | Set audience to `https://logic.azure.com` |
| **Easy Auth** | Production | Configure Azure AD on Logic App |
| **SAS URL** | Simple scenarios | Use signed callback URLs |

---

## Section 1: Setup and Configuration

In [ ]:
!pip install azure-ai-projects azure-ai-agents azure-identity --pre --quiet

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

# Foundry Configuration
PROJECT_ENDPOINT = os.getenv(
    "PROJECT_ENDPOINT",
    "https://ozgurguler-7212-resource.services.ai.azure.com/api/projects/ozgurguler-7212"
)

# Logic Apps MCP Server Configuration
# Replace with your Logic Apps MCP endpoint
MCP_SERVER_URL = os.getenv(
    "MCP_SERVER_URL",
    "https://your-logic-app.azurewebsites.net/api/mcpservers/ticketing/mcp"
)
MCP_SERVER_LABEL = "logic-apps-ticketing"

MODEL = os.getenv("MODEL_DEPLOYMENT_NAME", "gpt-5-nano")

print(f"Project: {PROJECT_ENDPOINT}")
print(f"MCP Server: {MCP_SERVER_URL}")

---

## Section 2: Create Agent with MCP Tool

### How MCP Tool Discovery Works

When you configure an `McpTool`, the following happens:

1. **Tool Discovery**: Agent queries the MCP server's `/mcp` endpoint
2. **Schema Retrieval**: Server returns available tools with their JSON schemas
3. **Tool Registration**: Tools are registered with the agent for invocation
4. **Runtime Execution**: When agent calls a tool, MCP protocol handles the request/response

### MCP Tool Configuration Options

```python
McpTool(
    server_url="https://...",      # MCP server endpoint
    server_label="my-tools",       # Identifier (alphanumeric + underscore)
    allowed_tools=[],              # Empty = all tools, or specify subset
    headers={"Authorization": ""}, # Custom headers for auth
)
```

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import McpTool

# Initialize client
client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# Configure MCP tool pointing to Logic Apps
# The server_label must be alphanumeric with underscores only
mcp_tool = McpTool(
    server_label=MCP_SERVER_LABEL.replace("-", "_"),  # Convert hyphens to underscores
    server_url=MCP_SERVER_URL,
    allowed_tools=[],  # Empty = allow all tools discovered from the server
)

print(f"MCP Tool configured:")
print(f"  Label: {MCP_SERVER_LABEL}")
print(f"  URL: {MCP_SERVER_URL}")
print(f"  Allowed tools: All")

In [ ]:
from azure.ai.projects.models import PromptAgentDefinition

AGENT_NAME = "logic-apps-mcp-agent"

# Agent instructions for enterprise tool usage
AGENT_INSTRUCTIONS = """You are a helpful assistant with access to enterprise tools via Logic Apps MCP server.

Available capabilities:
- Create, update, and close support tickets
- Send emails and notifications
- Query databases and CRM systems
- Manage documents and files

When asked to perform an action:
1. Identify the appropriate MCP tool to use
2. Gather required parameters from the user
3. Execute the tool and report results
4. Handle errors gracefully and suggest alternatives
"""

# Create agent with MCP tool
try:
    agent = client.agents.create_version(
        agent_name=AGENT_NAME,
        definition=PromptAgentDefinition(
            model=MODEL,
            instructions=AGENT_INSTRUCTIONS,
            tools=[mcp_tool],
        )
    )
    print(f"Created agent: {agent.name} (version: {agent.version})")
except Exception as e:
    print(f"Error creating agent: {e}")
    agent = None

---

## Section 3: Invoke Agent with MCP Tool

### The Tool Approval Flow

MCP tools require explicit approval before execution (security feature):

```
┌─────────────────────────────────────────────────────────────────┐
│                     MCP TOOL CALL FLOW                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  1. Agent decides to call MCP tool                               │
│                    │                                             │
│                    ▼                                             │
│  2. Run enters "requires_action" state                           │
│     (status: requires_action)                                    │
│                    │                                             │
│                    ▼                                             │
│  3. Application reviews tool call details                        │
│     - Tool name, parameters, target server                       │
│                    │                                             │
│                    ▼                                             │
│  4. Submit ToolApproval (approve=True/False)                     │
│                    │                                             │
│                    ▼                                             │
│  5. If approved: MCP server executes workflow                    │
│     If denied: Run fails with rejection                          │
│                    │                                             │
│                    ▼                                             │
│  6. Results returned to agent for next step                      │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### Why Approval Matters

| Concern | Protection |
|---------|------------|
| **Data Exfiltration** | Review what data is sent to external services |
| **Unauthorized Actions** | Prevent unintended modifications |
| **Cost Control** | Avoid unexpected API charges |
| **Audit Trail** | Log all approved/denied tool calls |

In [ ]:
# Get OpenAI client for agent invocation
openai_client = client.get_openai_client()

def invoke_mcp_agent(user_input: str, agent_name: str) -> str:
    """Send a request to the MCP-enabled agent."""
    print(f"\n{'='*60}")
    print(f"User: {user_input}")
    print("="*60)
    
    try:
        # Create a conversation
        conversation = openai_client.conversations.create()
        
        # Send the message with agent reference
        # Note: MCP tool calls may require approval in production
        response = openai_client.responses.create(
            input=user_input,
            conversation=conversation.id,
            extra_body={"agent": {"name": agent_name, "type": "agent_reference"}},
        )
        
        print(f"\nStatus: {response.status}")
        print(f"\nAgent Response:\n{response.output_text}")
        
        # Check if there were any tool calls
        if hasattr(response, 'tool_calls') and response.tool_calls:
            print(f"\nTool Calls Made:")
            for tc in response.tool_calls:
                print(f"  - {tc.name}: {tc.arguments}")
        
        return response.output_text
        
    except Exception as e:
        print(f"\nError: {e}")
        return None

# Test the agent with a sample request
if agent:
    result = invoke_mcp_agent(
        "Create a support ticket for 'Login issue on mobile app' with high priority",
        agent.name
    )
else:
    print("Agent not available - check MCP server configuration")

In [ ]:
# Additional test scenarios demonstrating enterprise tool capabilities
if agent:
    test_scenarios = [
        "What tools are available to me?",
        "Send an email to the IT team about scheduled maintenance tonight",
        "Check the status of ticket TKT-12345",
    ]
    
    for scenario in test_scenarios:
        result = invoke_mcp_agent(scenario, agent.name)
        print("\n")
else:
    print("Agent not available")

In [ ]:
# Cleanup - delete the agent
DELETE_AGENT = False  # Set to True to delete

if DELETE_AGENT and agent:
    try:
        client.agents.delete(agent_name=AGENT_NAME)
        print(f"Deleted agent: {AGENT_NAME}")
    except Exception as e:
        print(f"Error deleting agent: {e}")
else:
    print(f"Agent cleanup skipped (DELETE_AGENT = {DELETE_AGENT})")

---

## Section 5: Setting Up Logic Apps as MCP Server

### Step 1: Enable MCP in `host.json`

Add the following configuration to enable MCP server endpoints:

```json
{
  "version": "2.0",
  "extensions": {
    "workflow": {
      "McpServerEndpoints": {
        "enable": true
      }
    }
  }
}
```

### Step 2: Define MCP Servers in `mcpservers.json`

Create a file at the root of your Logic App to define available MCP servers:

```json
{
  "mcpServers": [
    {
      "name": "ticketing",
      "description": "IT Ticketing and Support System",
      "tools": [
        {
          "name": "CreateTicket",
          "description": "Create a new support ticket"
        },
        {
          "name": "GetTicketStatus",
          "description": "Get status of an existing ticket"
        },
        {
          "name": "CloseTicket",
          "description": "Close a resolved ticket"
        }
      ]
    },
    {
      "name": "crm",
      "description": "Customer Relationship Management",
      "tools": [
        {
          "name": "CreateContact",
          "description": "Create a new customer contact"
        },
        {
          "name": "UpdateOpportunity",
          "description": "Update sales opportunity"
        }
      ]
    }
  ]
}
```

### Step 3: Create Workflows for Each Tool

Each tool maps to a workflow with a **Request trigger**:

```
Logic App Structure:
├── host.json                 # MCP enabled
├── mcpservers.json          # Server definitions
├── CreateTicket/
│   └── workflow.json        # Request trigger → ServiceNow connector
├── GetTicketStatus/
│   └── workflow.json        # Request trigger → ServiceNow connector
├── CloseTicket/
│   └── workflow.json        # Request trigger → ServiceNow connector
└── ...
```

### Step 4: MCP Endpoint URL Pattern

```
https://<logic-app-name>.azurewebsites.net/api/mcpservers/<server-name>/mcp
```

Examples:
- `https://my-logic-app.azurewebsites.net/api/mcpservers/ticketing/mcp`
- `https://my-logic-app.azurewebsites.net/api/mcpservers/crm/mcp`

---

## Example Workflow: CreateTicket

Here's what a typical MCP tool workflow looks like:

```json
{
  "definition": {
    "$schema": "https://schema.management.azure.com/schemas/2016-06-01/workflowdefinition.json#",
    "triggers": {
      "When_a_HTTP_request_is_received": {
        "type": "Request",
        "kind": "Http",
        "inputs": {
          "schema": {
            "type": "object",
            "properties": {
              "title": { "type": "string" },
              "description": { "type": "string" },
              "priority": { "type": "string" }
            }
          }
        }
      }
    },
    "actions": {
      "Create_ServiceNow_Incident": {
        "type": "ApiConnection",
        "inputs": {
          "host": { "connection": { "name": "@parameters('$connections')['servicenow']['connectionId']" } },
          "method": "post",
          "path": "/api/now/table/incident",
          "body": {
            "short_description": "@triggerBody()?['title']",
            "description": "@triggerBody()?['description']",
            "urgency": "@triggerBody()?['priority']"
          }
        }
      },
      "Response": {
        "type": "Response",
        "inputs": {
          "statusCode": 200,
          "body": {
            "ticketId": "@body('Create_ServiceNow_Incident')?['result']?['sys_id']",
            "status": "created"
          }
        }
      }
    }
  }
}
```

---

## Enterprise Integration Scenarios

| Scenario | Logic Apps Connectors | MCP Tool |
|----------|----------------------|----------|
| **IT Service Management** | ServiceNow, Jira, Zendesk | CreateTicket, UpdateTicket, CloseTicket |
| **CRM Operations** | Salesforce, Dynamics 365, HubSpot | CreateLead, UpdateOpportunity, GetAccount |
| **HR Workflows** | Workday, SAP SuccessFactors | CreateEmployee, RequestTimeOff, GetPayslip |
| **ERP Integration** | SAP, Oracle, NetSuite | CreatePurchaseOrder, CheckInventory, UpdateInvoice |
| **Communication** | Office 365, Teams, Twilio | SendEmail, PostToChannel, SendSMS |
| **Document Management** | SharePoint, OneDrive, Box | UploadDocument, CreateFolder, ShareFile |
| **Approval Workflows** | Approvals, Power Automate | RequestApproval, GetApprovalStatus |

---

## Summary

### What Logic Apps + MCP Enables

```
┌────────────────────────────────────────────────────────────────────────────────┐
│                     LOGIC APPS AS MCP SERVER                                    │
├────────────────────────────────────────────────────────────────────────────────┤
│                                                                                 │
│   AI AGENT                    MCP PROTOCOL                   ENTERPRISE        │
│   ┌─────────┐                                                ┌─────────────┐   │
│   │ Foundry │  ──── Tool Discovery ────►  Logic Apps  ────►  │    SAP      │   │
│   │  Agent  │  ◄─── Tool Schemas ─────                       │ Salesforce  │   │
│   │         │  ──── Tool Calls ───────►  (1,400+     ────►   │ ServiceNow  │   │
│   │         │  ◄─── Results ──────────   Connectors)         │   SQL DB    │   │
│   └─────────┘                                                │ Office 365  │   │
│                                                              └─────────────┘   │
│                                                                                 │
│   Benefits:                                                                     │
│   • No code changes to existing enterprise integrations                        │
│   • Secure tool execution with approval workflows                              │
│   • Visual workflow designer for non-developers                                │
│   • Built-in retry, error handling, and monitoring                             │
│                                                                                 │
└────────────────────────────────────────────────────────────────────────────────┘
```

### Implementation Steps

| Step | Code/Action | Description |
|------|-------------|-------------|
| **1. Enable MCP** | `host.json` config | Enable MCP server endpoints |
| **2. Define Tools** | `mcpservers.json` | List available tools and descriptions |
| **3. Create Workflows** | Request trigger workflows | One workflow per MCP tool |
| **4. Configure Agent** | `McpTool(server_url=...)` | Point agent to MCP endpoint |
| **5. Handle Approvals** | `ToolApproval(approve=True)` | Approve/deny tool calls |

### Key Concepts

| Concept | Description |
|---------|-------------|
| **MCP Server** | Logic Apps endpoint exposing workflows as tools |
| **Tool Discovery** | Agent queries server for available tools |
| **Tool Schema** | JSON schema defining tool parameters |
| **Tool Approval** | Security checkpoint before execution |
| **Connector** | Logic Apps adapter to external service |

### Security Best Practices

| Practice | Implementation |
|----------|---------------|
| **Use Agent Identity** | Configure `AgenticIdentityToken` auth |
| **Limit Tool Scope** | Specify `allowed_tools` list |
| **Audit Tool Calls** | Log all approvals/rejections |
| **Network Isolation** | Use VNet integration for Logic Apps |
| **Secret Management** | Store credentials in Key Vault |

### Autonomous vs Conversational Agent Workflows

| Feature | Autonomous | Conversational |
|---------|------------|----------------|
| **Trigger** | HTTP, Schedule, Event | Chat session start |
| **Interaction** | No human input | Interactive chat |
| **Use Case** | Batch processing, automation | Support, helpdesk |
| **Context** | Workflow inputs | Chat history |
| **Output** | API response | Chat response |

---

## Next Steps

1. **Create your Logic App** with MCP enabled
2. **Build workflows** for your enterprise connectors
3. **Configure the agent** to use your MCP server
4. **Test with the Foundry Playground**

Continue to `../08-multi-agent-orchestration` for multi-agent patterns.

---

<div align="center">

## License & Attribution

This notebook is part of the **Azure AI Foundry Demo Repository**

[![License](https://img.shields.io/badge/License-Apache%202.0-blue.svg)](../LICENSE)

**Original Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub

**Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)

---

*If you use, modify, or distribute this work, you must provide appropriate credit to the original author as required by the [Apache License 2.0](../LICENSE).*

**Copyright © 2025 Ozgur Guler. All rights reserved.**

</div>